# 30 min

In [1]:
import pandas as pd
df = pd.read_csv("Full_data_30m.csv")

In [2]:
df

,Date_30m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,collapse,ATH,grow,rise,fall,date,open,close,Return,Return_sign
0,2022-01-01 00:30:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.769373,0.538746,0.324930,0.149739,20,46,...,1,17,7,13,42,2022-01-01 00:30:00,46375.42,46656.13,0.006053,1
1,2022-01-01 01:00:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.739433,0.478866,0.412354,0.175647,14,26,...,1,11,3,7,23,2022-01-01 01:00:00,46656.14,46815.33,0.003412,1
2,2022-01-01 01:30:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.768188,0.536376,0.349835,0.154206,15,17,...,0,8,10,7,19,2022-01-01 01:30:00,46815.34,46778.14,-0.000795,0
3,2022-01-01 02:00:00,"['2022-01-01 01:30:00', '2022-01-01 01:30:00',...","This year is my year , the year of my progress...",GhorbaniArshiya\nPopunderTraffic\nobjectwizard...,0.785896,0.571792,0.334061,0.200349,15,12,...,0,6,3,6,11,2022-01-01 02:00:00,46778.14,46826.01,0.001023,1
4,2022-01-01 02:30:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.774167,0.548335,0.403475,0.165008,13,13,...,0,10,5,6,16,2022-01-01 02:30:00,46826.00,46811.77,-0.000304,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2022-01-31 22:00:00,"['2022-01-31 21:30:00', '2022-01-31 21:30:00',...",#morganstanley calms tempers and minimizes #Bi...,Ricardo04606850\nTACryptowaves\ncryptonewsleak...,0.763569,0.527137,0.227818,0.145110,9,10,...,2,27,10,15,7,2022-01-31 22:00:00,38410.09,38547.30,0.003572,1
1484,2022-01-31 22:30:00,"['2022-01-31 22:00:00', '2022-01-31 22:00:00',...","BTC Latest Block Info: Block 721243 holds 2,80...",btc_blockbot\nM1Seahawk\nnialljburke\nmurray_r...,0.803204,0.606408,0.229487,0.144737,11,12,...,0,20,6,11,4,2022-01-31 22:30:00,38547.30,38412.46,-0.003498,0
1485,2022-01-31 23:00:00,"['2022-01-31 22:30:00', '2022-01-31 22:30:00',...","Good move of #Bitcoin today, reversal not conf...",CryptoMichNL\nIntEngineering\nKahunaATM\nMBrae...,0.775618,0.551236,0.206623,0.169323,3,11,...,1,17,4,5,12,2022-01-31 23:00:00,38412.45,38412.23,-0.000006,0
1486,2022-01-31 23:30:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.781215,0.562431,0.270270,0.161486,9,10,...,0,11,10,10,6,2022-01-31 23:30:00,38412.23,38466.90,0.001423,1


# Cross-validation

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
import pandas as pd

def cv_and_predict_valid(
    df_x,
    df_y,
    model,
    train_share=0.9,
    tree = False,
    grid_search=False,
    grid_parameters=None
):
    
    feature_importances = []

    split_index = int(train_share * len(df_x)) 
    X_train, X_valid = df_x[:split_index], df_x[split_index:]
    y_train, y_valid = df_y[:split_index], df_y[split_index:]
    
    
    # if scaler:
    #     scaler = MinMaxScaler()
    #     X_train["kk08_pos_score"] = scaler.fit_transform(X_train[["kk08_pos_score"]])   
    #     X_valid["kk08_pos_score"] = scaler.transform(X_valid[["kk08_pos_score"]]) 


    if grid_search and grid_parameters:
        grid_search_cv = GridSearchCV(
            estimator=model,
            param_grid=grid_parameters,
            cv=TimeSeriesSplit(n_splits=5),  
            scoring="balanced_accuracy",  
            n_jobs=-1        
        )
        grid_search_cv.fit(X_train, y_train)
        print(grid_search_cv.best_estimator_)
        model = grid_search_cv.best_estimator_  
        print(f"Best parameters from grid search: {grid_search_cv.best_params_}")
    else:
        model = deepcopy(model)
        model.fit(X_train, y_train)
    

    if tree:
        feature_importances = model.feature_importances_

    
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    roc_auc = roc_auc_score(y_valid, predictions)
    f1 = f1_score(y_valid, predictions)

    
    pred_df = pd.DataFrame()
    pred_df["target"] = y_valid
    pred_df["prediction"] = predictions

    
    print(f"Accuracy: {accuracy}")
    print(f"ROC-AUC: {roc_auc}")
    print(f"F1-score: {f1}")

    return pred_df, accuracy, roc_auc, f1, feature_importances


# Only kk08

In [4]:
df_x_1 = df[["kk08_pos_score"]]
df_y = df["Return_sign"]

In [5]:
train_share = 0.85
df_x_1_train = df_x_1.iloc[:int(len(df_x_1)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_1)*train_share)].reset_index(drop=True)

df_x_1_test = df_x_1.iloc[int(len(df_x_1)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_1)*train_share):].reset_index(drop=True)

In [6]:
sum(df_y_test)/len(df_y_test)

0.53125

In [7]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_1_train,df_y_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [8]:
# testing
dummy_model.fit(df_x_1_train, df_y_train)
predictions = dummy_model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {
    'n_estimators': [10, 30, 50, 100, 200],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 5, 10]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=50, random_state=17)
Best parameters from grid search: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 50}
Accuracy: 0.4251968503937008
ROC-AUC: 0.4407796101949026
F1-score: 0.3302752293577982


In [10]:
# testing
model = RandomForestClassifier(n_estimators=50,random_state=17)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5133928571428571
ROC-AUC: 0.5134453781512605
F1-score: 0.5281385281385281


In [11]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 20}
Accuracy: 0.48031496062992124
ROC-AUC: 0.49700149925037485
F1-score: 0.3888888888888889


In [12]:
# testing
model = XGBClassifier(max_depth=3, n_estimators=20, random_state=17)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.39285714285714285
ROC-AUC: 0.403921568627451
F1-score: 0.28421052631578947


In [13]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=2, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 2, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5511811023622047
ROC-AUC: 0.5086206896551724
F1-score: 0.7076923076923077


In [14]:
# testing
model = SVC(C=2, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.53125
ROC-AUC: 0.5
F1-score: 0.6938775510204082


In [15]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=100)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 100}
Accuracy: 0.4881889763779528
ROC-AUC: 0.5028735632183908
F1-score: 0.4144144144144144


In [16]:
# testing
model = AdaBoostClassifier(learning_rate=1, n_estimators=100)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.42410714285714285
ROC-AUC: 0.4394957983193277
F1-score: 0.26285714285714284


In [17]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, nb_model)


Accuracy: 0.4409448818897638
ROC-AUC: 0.44565217391304346
F1-score: 0.432


In [18]:
# testing
model = GaussianNB()
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.45535714285714285
ROC-AUC: 0.4521008403361345
F1-score: 0.49586776859504134


In [19]:
from sklearn.linear_model import LogisticRegression
grid_params = {
    'C': [0.01, 0.1, 1],           
    'penalty': ['l2']  
}
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, lr_model, grid_search=True, grid_parameters=grid_params)


LogisticRegression(C=1)
Best parameters from grid search: {'C': 1, 'penalty': 'l2'}
Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [20]:
# testing
model = LogisticRegression(C=1)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4732142857142857
ROC-AUC: 0.5042016806722689
F1-score: 0.016666666666666666


In [21]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5,6,7,8,9,10,15]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)
# meaningless


KNeighborsClassifier(n_neighbors=1)
Best parameters from grid search: {'n_neighbors': 1}
Accuracy: 0.4251968503937008
ROC-AUC: 0.4421539230384808
F1-score: 0.3177570093457944


In [22]:
knn_model = KNeighborsClassifier(n_neighbors=7)
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_1_train,df_y_train, knn_model)

Accuracy: 0.4251968503937008
ROC-AUC: 0.4311594202898551
F1-score: 0.4065040650406504


In [23]:
# testing
model = KNeighborsClassifier(n_neighbors=7)
model.fit(df_x_1_train, df_y_train)
predictions = model.predict(df_x_1_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4330357142857143
ROC-AUC: 0.43949579831932784
F1-score: 0.3864734299516908


# kk08 with lags

In [24]:
df['kk08_pos_score_lag1']=df['kk08_pos_score'].shift(1)
df['kk08_pos_score_lag2']=df['kk08_pos_score'].shift(2)
df['kk08_pos_score_lag3']=df['kk08_pos_score'].shift(3)


In [25]:
df_x_2 = df.dropna().reset_index(drop=True)[["kk08_pos_score", "kk08_pos_score_lag1", "kk08_pos_score_lag2", "kk08_pos_score_lag3"]]
df_y_2 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [26]:
df_x_2

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3
0,0.785896,0.768188,0.739433,0.769373
1,0.774167,0.785896,0.768188,0.739433
2,0.771805,0.774167,0.785896,0.768188
3,0.758940,0.771805,0.774167,0.785896
4,0.762820,0.758940,0.771805,0.774167
...,...,...,...,...
1480,0.763569,0.766771,0.758002,0.762439
1481,0.803204,0.763569,0.766771,0.758002
1482,0.775618,0.803204,0.763569,0.766771
1483,0.781215,0.775618,0.803204,0.763569


In [27]:
df_y_2

0       1
1       0
2       0
3       1
4       0
       ..
1480    1
1481    0
1482    0
1483    1
1484    0
Name: Return_sign, Length: 1485, dtype: int64

In [28]:
train_share = 0.85
df_x_2_train = df_x_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)
df_y_2_train = df_y_2.iloc[:int(len(df_x_2)*train_share)].reset_index(drop=True)

df_x_2_test = df_x_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)
df_y_2_test = df_y_2.iloc[int(len(df_x_2)*train_share):].reset_index(drop=True)

In [29]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_2_train,df_y_2_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [30]:
# testing
dummy_model.fit(df_x_2_train, df_y_2_train)
predictions = dummy_model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.47085201793721976
ROC-AUC: 0.5
F1-score: 0.0


In [31]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {
    'n_estimators': [10, 30, 50, 100, 200],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 5, 10]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(max_depth=3, min_samples_leaf=5, n_estimators=200,
                       random_state=17)
Best parameters from grid search: {'max_depth': 3, 'min_samples_leaf': 5, 'n_estimators': 200}
Accuracy: 0.5039370078740157
ROC-AUC: 0.5393553223388305
F1-score: 0.2222222222222222


In [32]:
# testing
model = RandomForestClassifier(max_depth=3, min_samples_leaf=5, n_estimators=200,
                       random_state=17)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4798206278026906
ROC-AUC: 0.48958837772397096
F1-score: 0.3958333333333333


In [33]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 20}
Accuracy: 0.48031496062992124
ROC-AUC: 0.517616191904048
F1-score: 0.15384615384615385


In [34]:
# testing
model = XGBClassifier(max_depth=1, n_estimators=20, random_state=17)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4798206278026906
ROC-AUC: 0.48958837772397096
F1-score: 0.3958333333333333


In [35]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5433070866141733
ROC-AUC: 0.5
F1-score: 0.7040816326530612


In [36]:
# testing
model = SVC(C=0.1, class_weight='balanced', kernel="poly")

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47085201793721976
ROC-AUC: 0.4679983857949959
F1-score: 0.5083333333333333


In [37]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=500)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 500}
Accuracy: 0.48031496062992124
ROC-AUC: 0.48463268365817097
F1-score: 0.47619047619047616


In [38]:
# testing
model = AdaBoostClassifier(learning_rate=1, n_estimators=500)
model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5650224215246636
ROC-AUC: 0.5643260694108152
F1-score: 0.5836909871244635


In [39]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, nb_model)


Accuracy: 0.49606299212598426
ROC-AUC: 0.47163918040979513
F1-score: 0.6190476190476191


In [40]:
# testing
model = GaussianNB()

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5112107623318386
ROC-AUC: 0.512953995157385
F1-score: 0.5112107623318386


In [41]:
from sklearn.linear_model import LogisticRegression
grid_params = {
    'C': [0.01, 0.1, 1],           
    'penalty': ['l2']  
}
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, lr_model, grid_search=True, grid_parameters=grid_params)


LogisticRegression(C=1)
Best parameters from grid search: {'C': 1, 'penalty': 'l2'}
Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [42]:
# testing
model = LogisticRegression(C=1)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4798206278026906
ROC-AUC: 0.5058514931396287
F1-score: 0.1076923076923077


In [43]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_2_train,df_y_2_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=10)
Best parameters from grid search: {'n_neighbors': 10}
Accuracy: 0.4881889763779528
ROC-AUC: 0.49875062468765613
F1-score: 0.4444444444444444


In [44]:
# testing
model = KNeighborsClassifier(n_neighbors=10)

model.fit(df_x_2_train, df_y_2_train)
predictions = model.predict(df_x_2_test)
accuracy = accuracy_score(df_y_2_test, predictions)
roc_auc = roc_auc_score(df_y_2_test, predictions)
f1 = f1_score(df_y_2_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.49327354260089684
ROC-AUC: 0.5043987086359968
F1-score: 0.39572192513368987


# Only vader

In [45]:
df_x_3 = df[["Sentiment_vader"]]
df_y = df["Return_sign"]

In [46]:
train_share = 0.85
df_x_3_train = df_x_3.iloc[:int(len(df_x_3)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_3)*train_share)].reset_index(drop=True)

df_x_3_test = df_x_3.iloc[int(len(df_x_3)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_3)*train_share):].reset_index(drop=True)

In [47]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_3_train,df_y_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [48]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [49]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50,100]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=50, random_state=17)
Best parameters from grid search: {'n_estimators': 50}
Accuracy: 0.5354330708661418
ROC-AUC: 0.539480259870065
F1-score: 0.5354330708661418


In [50]:
# testing
model = RandomForestClassifier(n_estimators=50,random_state=17)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4375
ROC-AUC: 0.4341736694677871
F1-score: 0.4793388429752066


In [51]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 6, 'n_estimators': 50}
Accuracy: 0.5196850393700787
ROC-AUC: 0.5194902548725637
F1-score: 0.5413533834586466


In [52]:
# testing
model = XGBClassifier(random_state=17, max_depth = 6, n_estimators = 50)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.45089285714285715
ROC-AUC: 0.45014005602240903
F1-score: 0.4721030042918455


In [53]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5433070866141733
ROC-AUC: 0.5
F1-score: 0.7040816326530612


In [54]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.53125
ROC-AUC: 0.5
F1-score: 0.6938775510204082


In [55]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


AdaBoostClassifier(learning_rate=0.3, n_estimators=10)
Best parameters from grid search: {'learning_rate': 0.3, 'n_estimators': 10}
Accuracy: 0.4330708661417323
ROC-AUC: 0.4700149925037481
F1-score: 0.07692307692307693


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [56]:
# testing
model = AdaBoostClassifier(learning_rate=0.3, n_estimators=10)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5178571428571429
ROC-AUC: 0.4997198879551821
F1-score: 0.6351351351351351


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [57]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, nb_model)


Accuracy: 0.5433070866141733
ROC-AUC: 0.5219890054972514
F1-score: 0.6463414634146342


In [58]:
# testing
model = GaussianNB()
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.5357142857142857
ROC-AUC: 0.5142857142857142
F1-score: 0.6623376623376623


In [59]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, lr_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [60]:
# testing
model = LogisticRegression()
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [61]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_3_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier()
Best parameters from grid search: {'n_neighbors': 5}
Accuracy: 0.5039370078740157
ROC-AUC: 0.5063718140929535
F1-score: 0.5116279069767442


In [62]:
# testing
model = KNeighborsClassifier(n_neighbors=5)
model.fit(df_x_3_train, df_y_train)
predictions = model.predict(df_x_3_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4375
ROC-AUC: 0.4392156862745098
F1-score: 0.4375


# Vader with lags

In [63]:
df

,Date_30m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,rise,fall,date,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3
0,2022-01-01 00:30:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.769373,0.538746,0.324930,0.149739,20,46,...,13,42,2022-01-01 00:30:00,46375.42,46656.13,0.006053,1,NaN,NaN,NaN
1,2022-01-01 01:00:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.739433,0.478866,0.412354,0.175647,14,26,...,7,23,2022-01-01 01:00:00,46656.14,46815.33,0.003412,1,0.769373,NaN,NaN
2,2022-01-01 01:30:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.768188,0.536376,0.349835,0.154206,15,17,...,7,19,2022-01-01 01:30:00,46815.34,46778.14,-0.000795,0,0.739433,0.769373,NaN
3,2022-01-01 02:00:00,"['2022-01-01 01:30:00', '2022-01-01 01:30:00',...","This year is my year , the year of my progress...",GhorbaniArshiya\nPopunderTraffic\nobjectwizard...,0.785896,0.571792,0.334061,0.200349,15,12,...,6,11,2022-01-01 02:00:00,46778.14,46826.01,0.001023,1,0.768188,0.739433,0.769373
4,2022-01-01 02:30:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.774167,0.548335,0.403475,0.165008,13,13,...,6,16,2022-01-01 02:30:00,46826.00,46811.77,-0.000304,0,0.785896,0.768188,0.739433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2022-01-31 22:00:00,"['2022-01-31 21:30:00', '2022-01-31 21:30:00',...",#morganstanley calms tempers and minimizes #Bi...,Ricardo04606850\nTACryptowaves\ncryptonewsleak...,0.763569,0.527137,0.227818,0.145110,9,10,...,15,7,2022-01-31 22:00:00,38410.09,38547.30,0.003572,1,0.766771,0.758002,0.762439
1484,2022-01-31 22:30:00,"['2022-01-31 22:00:00', '2022-01-31 22:00:00',...","BTC Latest Block Info: Block 721243 holds 2,80...",btc_blockbot\nM1Seahawk\nnialljburke\nmurray_r...,0.803204,0.606408,0.229487,0.144737,11,12,...,11,4,2022-01-31 22:30:00,38547.30,38412.46,-0.003498,0,0.763569,0.766771,0.758002
1485,2022-01-31 23:00:00,"['2022-01-31 22:30:00', '2022-01-31 22:30:00',...","Good move of #Bitcoin today, reversal not conf...",CryptoMichNL\nIntEngineering\nKahunaATM\nMBrae...,0.775618,0.551236,0.206623,0.169323,3,11,...,5,12,2022-01-31 23:00:00,38412.45,38412.23,-0.000006,0,0.803204,0.763569,0.766771
1486,2022-01-31 23:30:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.781215,0.562431,0.270270,0.161486,9,10,...,10,6,2022-01-31 23:30:00,38412.23,38466.90,0.001423,1,0.775618,0.803204,0.763569


In [64]:
df['Sentiment_vader_lag1']=df['Sentiment_vader'].shift(1)
df['Sentiment_vader_lag2']=df['Sentiment_vader'].shift(2)
df['Sentiment_vader_lag3']=df['Sentiment_vader'].shift(3)


In [65]:
df_x_4 = df.dropna().reset_index(drop=True)[["Sentiment_vader", "Sentiment_vader_lag1", "Sentiment_vader_lag2", "Sentiment_vader_lag3"]]
df_y_4 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [66]:
df_x_4

,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,0.200349,0.154206,0.175647,0.149739
1,0.165008,0.200349,0.154206,0.175647
2,0.200392,0.165008,0.200349,0.154206
3,0.164456,0.200392,0.165008,0.200349
4,0.186867,0.164456,0.200392,0.165008
...,...,...,...,...
1480,0.145110,0.140631,0.141404,0.130697
1481,0.144737,0.145110,0.140631,0.141404
1482,0.169323,0.144737,0.145110,0.140631
1483,0.161486,0.169323,0.144737,0.145110


In [67]:
df_y_4

0       1
1       0
2       0
3       1
4       0
       ..
1480    1
1481    0
1482    0
1483    1
1484    0
Name: Return_sign, Length: 1485, dtype: int64

In [68]:
train_share = 0.85
df_x_4_train = df_x_4.iloc[:int(len(df_x_4)*train_share)].reset_index(drop=True)
df_y_4_train = df_y_4.iloc[:int(len(df_x_4)*train_share)].reset_index(drop=True)

df_x_4_test = df_x_4.iloc[int(len(df_x_4)*train_share):].reset_index(drop=True)
df_y_4_test = df_y_4.iloc[int(len(df_x_4)*train_share):].reset_index(drop=True)

In [69]:
sum(df_y_4_test)/len(df_y_4_test)

0.5291479820627802

In [70]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_4_train,df_y_4_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [71]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=10, random_state=17)
Best parameters from grid search: {'n_estimators': 10}
Accuracy: 0.4566929133858268
ROC-AUC: 0.46426786606696646
F1-score: 0.4297520661157025


In [72]:
# testing
model = RandomForestClassifier(n_estimators=10, random_state=17)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.5381165919282511
ROC-AUC: 0.5425746569814367
F1-score: 0.5164319248826291


In [73]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 6, 'n_estimators': 300}
Accuracy: 0.41732283464566927
ROC-AUC: 0.418415792103948
F1-score: 0.4307692307692308


In [74]:
# testing
model = XGBClassifier(random_state=17, max_depth = 6, n_estimators= 300)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.4977578475336323
ROC-AUC: 0.4970944309927361
F1-score: 0.5172413793103449


In [75]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.5433070866141733
ROC-AUC: 0.5
F1-score: 0.7040816326530612


In [76]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')

model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47085201793721976
ROC-AUC: 0.5
F1-score: 0.0


In [77]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=500)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 500}
Accuracy: 0.4015748031496063
ROC-AUC: 0.40392303848075956
F1-score: 0.40625


In [78]:
# testing
model = model = AdaBoostClassifier(learning_rate=1, n_estimators=500)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5112107623318386
ROC-AUC: 0.5134786117836966
F1-score: 0.5067873303167421


In [79]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model)


Accuracy: 0.5118110236220472
ROC-AUC: 0.4916291854072964
F1-score: 0.6172839506172839


In [80]:
# testing
model = model = GaussianNB()
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad


Accuracy: 0.5381165919282511
ROC-AUC: 0.5205407586763519
F1-score: 0.6531986531986532


In [81]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.5)
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [82]:
# testing
model = model = LogisticRegression(C=0.5)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47085201793721976
ROC-AUC: 0.5
F1-score: 0.0


In [83]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_4_train,df_y_4_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=10)
Best parameters from grid search: {'n_neighbors': 10}
Accuracy: 0.48031496062992124
ROC-AUC: 0.4928785607196402
F1-score: 0.42105263157894735


In [84]:
# testing
model = model = KNeighborsClassifier(n_neighbors=10)
model.fit(df_x_4_train, df_y_4_train)
predictions = model.predict(df_x_4_test)
accuracy = accuracy_score(df_y_4_test, predictions)
roc_auc = roc_auc_score(df_y_4_test, predictions)
f1 = f1_score(df_y_4_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.5695067264573991
ROC-AUC: 0.5795803066989508
F1-score: 0.5


# Only elkulako

In [85]:
df

,Date_30m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,2022-01-01 00:30:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.769373,0.538746,0.324930,0.149739,20,46,...,46375.42,46656.13,0.006053,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.739433,0.478866,0.412354,0.175647,14,26,...,46656.14,46815.33,0.003412,1,0.769373,NaN,NaN,0.149739,NaN,NaN
2,2022-01-01 01:30:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.768188,0.536376,0.349835,0.154206,15,17,...,46815.34,46778.14,-0.000795,0,0.739433,0.769373,NaN,0.175647,0.149739,NaN
3,2022-01-01 02:00:00,"['2022-01-01 01:30:00', '2022-01-01 01:30:00',...","This year is my year , the year of my progress...",GhorbaniArshiya\nPopunderTraffic\nobjectwizard...,0.785896,0.571792,0.334061,0.200349,15,12,...,46778.14,46826.01,0.001023,1,0.768188,0.739433,0.769373,0.154206,0.175647,0.149739
4,2022-01-01 02:30:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.774167,0.548335,0.403475,0.165008,13,13,...,46826.00,46811.77,-0.000304,0,0.785896,0.768188,0.739433,0.200349,0.154206,0.175647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2022-01-31 22:00:00,"['2022-01-31 21:30:00', '2022-01-31 21:30:00',...",#morganstanley calms tempers and minimizes #Bi...,Ricardo04606850\nTACryptowaves\ncryptonewsleak...,0.763569,0.527137,0.227818,0.145110,9,10,...,38410.09,38547.30,0.003572,1,0.766771,0.758002,0.762439,0.140631,0.141404,0.130697
1484,2022-01-31 22:30:00,"['2022-01-31 22:00:00', '2022-01-31 22:00:00',...","BTC Latest Block Info: Block 721243 holds 2,80...",btc_blockbot\nM1Seahawk\nnialljburke\nmurray_r...,0.803204,0.606408,0.229487,0.144737,11,12,...,38547.30,38412.46,-0.003498,0,0.763569,0.766771,0.758002,0.145110,0.140631,0.141404
1485,2022-01-31 23:00:00,"['2022-01-31 22:30:00', '2022-01-31 22:30:00',...","Good move of #Bitcoin today, reversal not conf...",CryptoMichNL\nIntEngineering\nKahunaATM\nMBrae...,0.775618,0.551236,0.206623,0.169323,3,11,...,38412.45,38412.23,-0.000006,0,0.803204,0.763569,0.766771,0.144737,0.145110,0.140631
1486,2022-01-31 23:30:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.781215,0.562431,0.270270,0.161486,9,10,...,38412.23,38466.90,0.001423,1,0.775618,0.803204,0.763569,0.169323,0.144737,0.145110


In [86]:
df_x_5 = df[["Elkulako_label"]]
df_y = df["Return_sign"]

In [87]:
df_x_5

,Elkulako_label
0,0.324930
1,0.412354
2,0.349835
3,0.334061
4,0.403475
...,...
1483,0.227818
1484,0.229487
1485,0.206623
1486,0.270270


In [88]:
train_share = 0.85
df_x_5_train = df_x_5.iloc[:int(len(df_x_5)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_x_5)*train_share)].reset_index(drop=True)

df_x_5_test = df_x_5.iloc[int(len(df_x_5)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_x_5)*train_share):].reset_index(drop=True)

In [89]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_5_train,df_y_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [90]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [91]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50,100]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=5, random_state=17)
Best parameters from grid search: {'n_estimators': 5}
Accuracy: 0.5590551181102362
ROC-AUC: 0.5625937031484258
F1-score: 0.5625


In [92]:
# testing
model = RandomForestClassifier(n_estimators=5, random_state=17)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# very good

Accuracy: 0.5669642857142857
ROC-AUC: 0.5689075630252101
F1-score: 0.5688888888888889


In [93]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)
#bad

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 20}
Accuracy: 0.49606299212598426
ROC-AUC: 0.5018740629685158
F1-score: 0.4838709677419355


In [94]:
xgb_model = XGBClassifier(random_state=17, max_depth = 6, n_estimators = 300)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, xgb_model, tree = True)

Accuracy: 0.5196850393700787
ROC-AUC: 0.5194902548725637
F1-score: 0.5413533834586466


In [95]:
# testing
model = XGBClassifier(random_state=17, max_depth = 6, n_estimators = 300)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.5803571428571429
ROC-AUC: 0.5815126050420169
F1-score: 0.5877192982456141


In [96]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1, 2],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, svm_model, grid_search=True, grid_parameters=grid_params)

SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [97]:
# testing
model = SVC(C=0.1, class_weight='balanced', gamma=1, kernel='poly')
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.53125
ROC-AUC: 0.5
F1-score: 0.6938775510204082


In [98]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.3, n_estimators=300)
Best parameters from grid search: {'learning_rate': 0.3, 'n_estimators': 300}
Accuracy: 0.49606299212598426
ROC-AUC: 0.5059970014992504
F1-score: 0.4576271186440678


In [99]:
# testing
model = AdaBoostClassifier(learning_rate=0.3, n_estimators=300)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.53125
ROC-AUC: 0.5476190476190476
F1-score: 0.3930635838150289


In [100]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, nb_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [101]:
# testing
model = GaussianNB()
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [102]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, lr_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [103]:
# testing
model = LogisticRegression()
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [104]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_5_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier()
Best parameters from grid search: {'n_neighbors': 5}
Accuracy: 0.5275590551181102
ROC-AUC: 0.5322338830584707
F1-score: 0.5238095238095238


In [105]:
# testing
model = KNeighborsClassifier(n_neighbors=5)
model.fit(df_x_5_train, df_y_train)
predictions = model.predict(df_x_5_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.5223214285714286
ROC-AUC: 0.5257703081232493
F1-score: 0.5114155251141552


# Elkulako with lags

In [106]:
df

,Date_30m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,open,close,Return,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3
0,2022-01-01 00:30:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.769373,0.538746,0.324930,0.149739,20,46,...,46375.42,46656.13,0.006053,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.739433,0.478866,0.412354,0.175647,14,26,...,46656.14,46815.33,0.003412,1,0.769373,NaN,NaN,0.149739,NaN,NaN
2,2022-01-01 01:30:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.768188,0.536376,0.349835,0.154206,15,17,...,46815.34,46778.14,-0.000795,0,0.739433,0.769373,NaN,0.175647,0.149739,NaN
3,2022-01-01 02:00:00,"['2022-01-01 01:30:00', '2022-01-01 01:30:00',...","This year is my year , the year of my progress...",GhorbaniArshiya\nPopunderTraffic\nobjectwizard...,0.785896,0.571792,0.334061,0.200349,15,12,...,46778.14,46826.01,0.001023,1,0.768188,0.739433,0.769373,0.154206,0.175647,0.149739
4,2022-01-01 02:30:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.774167,0.548335,0.403475,0.165008,13,13,...,46826.00,46811.77,-0.000304,0,0.785896,0.768188,0.739433,0.200349,0.154206,0.175647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2022-01-31 22:00:00,"['2022-01-31 21:30:00', '2022-01-31 21:30:00',...",#morganstanley calms tempers and minimizes #Bi...,Ricardo04606850\nTACryptowaves\ncryptonewsleak...,0.763569,0.527137,0.227818,0.145110,9,10,...,38410.09,38547.30,0.003572,1,0.766771,0.758002,0.762439,0.140631,0.141404,0.130697
1484,2022-01-31 22:30:00,"['2022-01-31 22:00:00', '2022-01-31 22:00:00',...","BTC Latest Block Info: Block 721243 holds 2,80...",btc_blockbot\nM1Seahawk\nnialljburke\nmurray_r...,0.803204,0.606408,0.229487,0.144737,11,12,...,38547.30,38412.46,-0.003498,0,0.763569,0.766771,0.758002,0.145110,0.140631,0.141404
1485,2022-01-31 23:00:00,"['2022-01-31 22:30:00', '2022-01-31 22:30:00',...","Good move of #Bitcoin today, reversal not conf...",CryptoMichNL\nIntEngineering\nKahunaATM\nMBrae...,0.775618,0.551236,0.206623,0.169323,3,11,...,38412.45,38412.23,-0.000006,0,0.803204,0.763569,0.766771,0.144737,0.145110,0.140631
1486,2022-01-31 23:30:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.781215,0.562431,0.270270,0.161486,9,10,...,38412.23,38466.90,0.001423,1,0.775618,0.803204,0.763569,0.169323,0.144737,0.145110


In [107]:
df['Elkulako_label_lag1']=df['Elkulako_label'].shift(1)
df['Elkulako_label_lag2']=df['Elkulako_label'].shift(2)
df['Elkulako_label_lag3']=df['Elkulako_label'].shift(3)


In [108]:
df_x_6 = df.dropna().reset_index(drop=True)[["Elkulako_label", "Elkulako_label_lag1", "Elkulako_label_lag2", "Elkulako_label_lag3"]]
df_y_6 = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [109]:
df_x_6

,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.334061,0.349835,0.412354,0.324930
1,0.403475,0.334061,0.349835,0.412354
2,0.455128,0.403475,0.334061,0.349835
3,0.321212,0.455128,0.403475,0.334061
4,0.329596,0.321212,0.455128,0.403475
...,...,...,...,...
1480,0.227818,0.260274,0.291310,0.280872
1481,0.229487,0.227818,0.260274,0.291310
1482,0.206623,0.229487,0.227818,0.260274
1483,0.270270,0.206623,0.229487,0.227818


In [110]:
df_y_6

0       1
1       0
2       0
3       1
4       0
       ..
1480    1
1481    0
1482    0
1483    1
1484    0
Name: Return_sign, Length: 1485, dtype: int64

In [111]:
train_share = 0.85
df_x_6_train = df_x_6.iloc[:int(len(df_x_6)*train_share)].reset_index(drop=True)
df_y_6_train = df_y_6.iloc[:int(len(df_x_6)*train_share)].reset_index(drop=True)

df_x_6_test = df_x_6.iloc[int(len(df_x_6)*train_share):].reset_index(drop=True)
df_y_6_test = df_y_6.iloc[int(len(df_x_6)*train_share):].reset_index(drop=True)

In [112]:
sum(df_y_6_test)/len(df_y_6_test)

0.5291479820627802

In [113]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_6_train,df_y_6_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [114]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=300, random_state=17)
Best parameters from grid search: {'n_estimators': 300}
Accuracy: 0.5275590551181102
ROC-AUC: 0.5281109445277362
F1-score: 0.5454545454545454


In [115]:
# testing
model = RandomForestClassifier(n_estimators=300, random_state=17)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4977578475336323
ROC-AUC: 0.500766747376917
F1-score: 0.48623853211009177


In [116]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 1, 'n_estimators': 100}
Accuracy: 0.4566929133858268
ROC-AUC: 0.4628935532233884
F1-score: 0.43902439024390244


In [117]:
# testing
model = XGBClassifier(random_state=17, max_depth = 1, n_estimators= 100)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# not bad

Accuracy: 0.4977578475336323
ROC-AUC: 0.5012913640032284
F1-score: 0.48148148148148145


In [118]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=1, class_weight='balanced', gamma=1)
Best parameters from grid search: {'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [119]:
# testing
model = SVC(C=1, class_weight='balanced', gamma=1)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.47085201793721976
ROC-AUC: 0.5
F1-score: 0.0


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=500)
Best parameters from grid search: {'learning_rate': 1, 'n_estimators': 500}
Accuracy: 0.5669291338582677
ROC-AUC: 0.5725887056471765
F1-score: 0.56


In [ ]:
# testing
model = AdaBoostClassifier(learning_rate=1, n_estimators=500)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5695067264573991
ROC-AUC: 0.5727602905569007
F1-score: 0.5596330275229358


In [122]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [ ]:
# testing
model = GaussianNB()
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.47085201793721976
ROC-AUC: 0.5
F1-score: 0.0


In [124]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.49725137431284355
F1-score: 0.0547945205479452


In [ ]:
# testing
model = LogisticRegression()
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.47533632286995514
ROC-AUC: 0.4937449556093624
F1-score: 0.2641509433962264


In [126]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_6_train,df_y_6_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=10)
Best parameters from grid search: {'n_neighbors': 10}
Accuracy: 0.5196850393700787
ROC-AUC: 0.534607696151924
F1-score: 0.45045045045045046


In [ ]:
# testing
model = KNeighborsClassifier(n_neighbors=10)
model.fit(df_x_6_train, df_y_6_train)
predictions = model.predict(df_x_6_test)
accuracy = accuracy_score(df_y_6_test, predictions)
roc_auc = roc_auc_score(df_y_6_test, predictions)
f1 = f1_score(df_y_6_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.48878923766816146
ROC-AUC: 0.4954398708635997
F1-score: 0.4411764705882353


# All sentiments with lags

In [128]:
df

,Date_30m,datetime,text,username,kk08_pos_score,kk08_score_diff,Elkulako_label,Sentiment_vader,pred,analysis,...,Return_sign,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,2022-01-01 00:30:00,"['2022-01-01 00:00:00', '2022-01-01 00:00:00',...",Current Price of Bitcoin:\n$46320 (-1.85%)\n\n...,CryptoNerdApp\nTrendSpider\nMadStudentScie1\nW...,0.769373,0.538746,0.324930,0.149739,20,46,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,"['2022-01-01 00:30:00', '2022-01-01 00:30:01',...",“Here's to a bright New Year and a fond farewe...,bitcoiva\ncryptozone007\nlinuxadvocate\nbakedn...,0.739433,0.478866,0.412354,0.175647,14,26,...,1,0.769373,NaN,NaN,0.149739,NaN,NaN,0.324930,NaN,NaN
2,2022-01-01 01:30:00,"['2022-01-01 01:00:00', '2022-01-01 01:00:00',...",Today's Cryptocurrency Fear And Greed Index: 2...,FearAndGreedBot\nOfficialOverbit\nBTC_AI_bot\n...,0.768188,0.536376,0.349835,0.154206,15,17,...,0,0.739433,0.769373,NaN,0.175647,0.149739,NaN,0.412354,0.324930,NaN
3,2022-01-01 02:00:00,"['2022-01-01 01:30:00', '2022-01-01 01:30:00',...","This year is my year , the year of my progress...",GhorbaniArshiya\nPopunderTraffic\nobjectwizard...,0.785896,0.571792,0.334061,0.200349,15,12,...,1,0.768188,0.739433,0.769373,0.154206,0.175647,0.149739,0.349835,0.412354,0.324930
4,2022-01-01 02:30:00,"['2022-01-01 02:00:00', '2022-01-01 02:00:00',...",Huobi Singapore would like to wish you a Happy...,HuobiSg\nAmazonCash4Cars\nBTC_AI_bot\nCalvinAy...,0.774167,0.548335,0.403475,0.165008,13,13,...,0,0.785896,0.768188,0.739433,0.200349,0.154206,0.175647,0.334061,0.349835,0.412354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2022-01-31 22:00:00,"['2022-01-31 21:30:00', '2022-01-31 21:30:00',...",#morganstanley calms tempers and minimizes #Bi...,Ricardo04606850\nTACryptowaves\ncryptonewsleak...,0.763569,0.527137,0.227818,0.145110,9,10,...,1,0.766771,0.758002,0.762439,0.140631,0.141404,0.130697,0.260274,0.291310,0.280872
1484,2022-01-31 22:30:00,"['2022-01-31 22:00:00', '2022-01-31 22:00:00',...","BTC Latest Block Info: Block 721243 holds 2,80...",btc_blockbot\nM1Seahawk\nnialljburke\nmurray_r...,0.803204,0.606408,0.229487,0.144737,11,12,...,0,0.763569,0.766771,0.758002,0.145110,0.140631,0.141404,0.227818,0.260274,0.291310
1485,2022-01-31 23:00:00,"['2022-01-31 22:30:00', '2022-01-31 22:30:00',...","Good move of #Bitcoin today, reversal not conf...",CryptoMichNL\nIntEngineering\nKahunaATM\nMBrae...,0.775618,0.551236,0.206623,0.169323,3,11,...,0,0.803204,0.763569,0.766771,0.144737,0.145110,0.140631,0.229487,0.227818,0.260274
1486,2022-01-31 23:30:00,"['2022-01-31 23:00:00', '2022-01-31 23:00:00',...",Top #Cryptocurrency RANK DOWN (#crypto #bitcoi...,CryptoTopCharts\nBTC_AI_bot\ncryptonewsleak\nt...,0.781215,0.562431,0.270270,0.161486,9,10,...,1,0.775618,0.803204,0.763569,0.169323,0.144737,0.145110,0.206623,0.229487,0.227818


In [129]:
pd.concat([df_x_2, df_x_4, df_x_6], axis =1)

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.785896,0.768188,0.739433,0.769373,0.200349,0.154206,0.175647,0.149739,0.334061,0.349835,0.412354,0.324930
1,0.774167,0.785896,0.768188,0.739433,0.165008,0.200349,0.154206,0.175647,0.403475,0.334061,0.349835,0.412354
2,0.771805,0.774167,0.785896,0.768188,0.200392,0.165008,0.200349,0.154206,0.455128,0.403475,0.334061,0.349835
3,0.758940,0.771805,0.774167,0.785896,0.164456,0.200392,0.165008,0.200349,0.321212,0.455128,0.403475,0.334061
4,0.762820,0.758940,0.771805,0.774167,0.186867,0.164456,0.200392,0.165008,0.329596,0.321212,0.455128,0.403475
...,...,...,...,...,...,...,...,...,...,...,...,...
1480,0.763569,0.766771,0.758002,0.762439,0.145110,0.140631,0.141404,0.130697,0.227818,0.260274,0.291310,0.280872
1481,0.803204,0.763569,0.766771,0.758002,0.144737,0.145110,0.140631,0.141404,0.229487,0.227818,0.260274,0.291310
1482,0.775618,0.803204,0.763569,0.766771,0.169323,0.144737,0.145110,0.140631,0.206623,0.229487,0.227818,0.260274
1483,0.781215,0.775618,0.803204,0.763569,0.161486,0.169323,0.144737,0.145110,0.270270,0.206623,0.229487,0.227818


In [130]:
df_x_all = pd.concat([df_x_2, df_x_4, df_x_6], axis =1)
df_y_all = df["Return_sign"].iloc[3:].reset_index(drop=True)

In [131]:
df_x_all

,kk08_pos_score,kk08_pos_score_lag1,kk08_pos_score_lag2,kk08_pos_score_lag3,Sentiment_vader,Sentiment_vader_lag1,Sentiment_vader_lag2,Sentiment_vader_lag3,Elkulako_label,Elkulako_label_lag1,Elkulako_label_lag2,Elkulako_label_lag3
0,0.785896,0.768188,0.739433,0.769373,0.200349,0.154206,0.175647,0.149739,0.334061,0.349835,0.412354,0.324930
1,0.774167,0.785896,0.768188,0.739433,0.165008,0.200349,0.154206,0.175647,0.403475,0.334061,0.349835,0.412354
2,0.771805,0.774167,0.785896,0.768188,0.200392,0.165008,0.200349,0.154206,0.455128,0.403475,0.334061,0.349835
3,0.758940,0.771805,0.774167,0.785896,0.164456,0.200392,0.165008,0.200349,0.321212,0.455128,0.403475,0.334061
4,0.762820,0.758940,0.771805,0.774167,0.186867,0.164456,0.200392,0.165008,0.329596,0.321212,0.455128,0.403475
...,...,...,...,...,...,...,...,...,...,...,...,...
1480,0.763569,0.766771,0.758002,0.762439,0.145110,0.140631,0.141404,0.130697,0.227818,0.260274,0.291310,0.280872
1481,0.803204,0.763569,0.766771,0.758002,0.144737,0.145110,0.140631,0.141404,0.229487,0.227818,0.260274,0.291310
1482,0.775618,0.803204,0.763569,0.766771,0.169323,0.144737,0.145110,0.140631,0.206623,0.229487,0.227818,0.260274
1483,0.781215,0.775618,0.803204,0.763569,0.161486,0.169323,0.144737,0.145110,0.270270,0.206623,0.229487,0.227818


In [132]:
df_y_6

0       1
1       0
2       0
3       1
4       0
       ..
1480    1
1481    0
1482    0
1483    1
1484    0
Name: Return_sign, Length: 1485, dtype: int64

In [133]:
train_share = 0.85
df_x_all_train = df_x_all.iloc[:int(len(df_x_all)*train_share)].reset_index(drop=True)
df_y_all_train = df_y_all.iloc[:int(len(df_x_all)*train_share)].reset_index(drop=True)

df_x_all_test = df_x_all.iloc[int(len(df_x_all)*train_share):].reset_index(drop=True)
df_y_all_test = df_y_all.iloc[int(len(df_x_all)*train_share):].reset_index(drop=True)

In [134]:
sum(df_y_all_test)/len(df_y_all_test)

0.5291479820627802

In [135]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_all_train,df_y_all_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [136]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [5, 10, 20, 50, 100, 300, 500]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(n_estimators=20, random_state=17)
Best parameters from grid search: {'n_estimators': 20}
Accuracy: 0.5590551181102362
ROC-AUC: 0.5625937031484258
F1-score: 0.5625


In [137]:
# testing
model = RandomForestClassifier(n_estimators=20,random_state=17)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.49327354260089684
ROC-AUC: 0.49862792574656983
F1-score: 0.45933014354066987


In [138]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [100, 300, 500],"max_depth": [1,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 3, 'n_estimators': 300}
Accuracy: 0.5196850393700787
ROC-AUC: 0.5153673163418291
F1-score: 0.5611510791366906


In [139]:
# testing
model = XGBClassifier(random_state=17, max_depth = 3, n_estimators= 300)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.48878923766816146
ROC-AUC: 0.4865213882163034
F1-score: 0.5210084033613446


In [140]:
from sklearn.svm import SVC
# SVM
grid_params = {'C': [0.1, 0.3, 0.7, 1],  'kernel': ['poly', 'rbf'], 'gamma': [1, 0.1, 0.01], 'class_weight': ['balanced', None]}
svm_model = SVC()
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, svm_model, grid_search=True, grid_parameters=grid_params)
# too bad

SVC(C=0.3, gamma=1, kernel='poly')
Best parameters from grid search: {'C': 0.3, 'class_weight': None, 'gamma': 1, 'kernel': 'poly'}
Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [141]:
# testing
model = SVC(C=0.3, kernel="poly", gamma=1)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.4663677130044843
ROC-AUC: 0.4936642453591606
F1-score: 0.048


In [142]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model, tree = True, grid_search=True, grid_parameters = grid_params)
# too bad

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.1)
Best parameters from grid search: {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy: 0.4566929133858268
ROC-AUC: 0.4848825587206397
F1-score: 0.24175824175824176


In [143]:
model = AdaBoostClassifier(n_estimators=500)
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model, tree = True)

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5039370078740157
ROC-AUC: 0.5063718140929535
F1-score: 0.5116279069767442


In [144]:
# testing
model = AdaBoostClassifier(n_estimators=500)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.5201793721973094
ROC-AUC: 0.5214285714285715
F1-score: 0.5244444444444445


In [145]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model)


Accuracy: 0.4330708661417323
ROC-AUC: 0.4617691154422789
F1-score: 0.2


In [146]:
# testing
model  = GaussianNB()
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.4977578475336323
ROC-AUC: 0.5180790960451978
F1-score: 0.2631578947368421


In [147]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_all_train,df_y_all_train, model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=10)
Best parameters from grid search: {'n_neighbors': 10}
Accuracy: 0.4645669291338583
ROC-AUC: 0.4825087456271864
F1-score: 0.3584905660377358


In [ ]:
# testing
model = KNeighborsClassifier(n_neighbors=10)
model.fit(df_x_all_train, df_y_all_train)
predictions = model.predict(df_x_all_test)
accuracy = accuracy_score(df_y_all_test, predictions)
roc_auc = roc_auc_score(df_y_all_test, predictions)
f1 = f1_score(df_y_all_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.515695067264574
ROC-AUC: 0.5261097659402744
F1-score: 0.43157894736842106


# With dummies

In [149]:
df.columns

Index(['Date_30m', 'datetime', 'text', 'username', 'kk08_pos_score',
       'kk08_score_diff', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall', 'date', 'open', 'close', 'Return', 'Return_sign',
       'kk08_pos_score_lag1', 'kk08_pos_score_lag2', 'kk08_pos_score_lag3',
       'Sentiment_vader_lag1', 'Sentiment_vader_lag2', 'Sentiment_vader_lag3',
       'Elkulako_label_lag1', 'Elkulako_label_lag2', 'Elkulako_label_lag3'],
      dtype='object')

In [150]:
df_x_dum = df[['kk08_pos_score', 'Elkulako_label', 'Sentiment_vader', 'pred',
       'analysis', 'short', 'long', 'buy', 'sell', 'bull', 'bear', 'bankrupt',
       'fraud', 'cbdc', 'record', 'crisis', 'collapse', 'ATH', 'grow', 'rise',
       'fall']]
df_y = df["Return_sign"]

In [151]:
df_x_dum

,kk08_pos_score,Elkulako_label,Sentiment_vader,pred,analysis,short,long,buy,sell,bull,...,bankrupt,fraud,cbdc,record,crisis,collapse,ATH,grow,rise,fall
0,0.769373,0.324930,0.149739,20,46,45,20,49,26,14,...,0,1,0,1,0,1,17,7,13,42
1,0.739433,0.412354,0.175647,14,26,26,11,38,11,9,...,0,0,1,3,0,1,11,3,7,23
2,0.768188,0.349835,0.154206,15,17,21,11,47,20,18,...,0,0,1,2,0,0,8,10,7,19
3,0.785896,0.334061,0.200349,15,12,11,8,26,10,4,...,0,0,0,8,0,0,6,3,6,11
4,0.774167,0.403475,0.165008,13,13,14,7,37,19,7,...,0,2,0,2,0,0,10,5,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,0.763569,0.227818,0.145110,9,10,5,22,43,21,15,...,0,3,1,4,0,2,27,10,15,7
1484,0.803204,0.229487,0.144737,11,12,10,24,53,16,14,...,0,0,0,2,0,0,20,6,11,4
1485,0.775618,0.206623,0.169323,3,11,12,16,39,18,17,...,0,1,3,2,1,1,17,4,5,12
1486,0.781215,0.270270,0.161486,9,10,11,21,46,14,21,...,0,1,0,1,0,0,11,10,10,6


In [152]:
train_share = 0.85
df_x_dum_train = df_x_dum.iloc[:int(len(df_x_dum)*train_share)].reset_index(drop=True)
df_y_train = df_y.iloc[:int(len(df_y)*train_share)].reset_index(drop=True)

df_x_dum_test = df_x_dum.iloc[int(len(df_x_dum)*train_share):].reset_index(drop=True)
df_y_test = df_y.iloc[int(len(df_y)*train_share):].reset_index(drop=True)

In [153]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_pred, dummy_acc, dummy_roc_auc,dummy_f1, dummy_feat_imp= cv_and_predict_valid(df_x_dum_train,df_y_train,dummy_model)


Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [154]:
# testing
model = DummyClassifier(strategy='most_frequent')

model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [155]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest
grid_params = {'n_estimators': [50,100, 300 ,500, 700]}
rf_model = RandomForestClassifier(random_state=17)
rf_pred, rf_acc, rf_roc_auc,rf_f1, rf_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, rf_model, tree = True, grid_search=True, grid_parameters=grid_params)

RandomForestClassifier(random_state=17)
Best parameters from grid search: {'n_estimators': 100}
Accuracy: 0.5039370078740157
ROC-AUC: 0.5022488755622189
F1-score: 0.5333333333333333


In [156]:
# testing
model = RandomForestClassifier(random_state=17)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")


Accuracy: 0.5044642857142857
ROC-AUC: 0.5106442577030812
F1-score: 0.4688995215311005


In [157]:
from xgboost import XGBClassifier
# XGBoost
grid_params = {'n_estimators': [50, 100, 300, 500, 1000],"max_depth": [1,2,3,6]}
xgb_model = XGBClassifier(random_state=17)
xgb_pred, xgb_acc, xgb_roc_auc,xgb_f1, xgb_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, xgb_model, tree = True, grid_search=True, grid_parameters=grid_params)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)
Best parameters from grid search: {'max_depth': 6, 'n_estimators': 50}
Accuracy: 0.49606299212598426
ROC-AUC: 0.4936281859070465
F1-score: 0.5294117647058824


In [158]:
# testing
model = XGBClassifier(random_state=17, max_depth = 6, n_estimators = 50)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.4732142857142857
ROC-AUC: 0.4761904761904762
F1-score: 0.4636363636363636


In [159]:
from sklearn.svm import SVC
# SVM
svm_model = SVC(kernel="poly", C = 0.5)
svm_pred, svm_acc, svm_roc_auc,svm_f1, svm_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, svm_model)

Accuracy: 0.4566929133858268
ROC-AUC: 0.5
F1-score: 0.0


In [ ]:
# testing
model = SVC(kernel="poly", C = 0.5)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.46875
ROC-AUC: 0.5
F1-score: 0.0


In [161]:
from sklearn.ensemble import AdaBoostClassifier
grid_params = {'n_estimators': [5, 10, 20, 30, 50, 100, 300, 500], 'learning_rate':[0.1, 0.3, 0.5, 1]}
ada_model = AdaBoostClassifier()
ada_pred, ada_acc, ada_roc_auc,ada_f1, ada_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, ada_model, tree = True, grid_search=True, grid_parameters = grid_params)


c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.5, n_estimators=500)
Best parameters from grid search: {'learning_rate': 0.5, 'n_estimators': 500}
Accuracy: 0.5196850393700787
ROC-AUC: 0.5236131934032984
F1-score: 0.5196850393700787


In [162]:
# testing
model = AdaBoostClassifier(learning_rate=0.5, n_estimators=500)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

c:\Users\isaen\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.46875
ROC-AUC: 0.47086834733893557
F1-score: 0.4663677130044843


In [163]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_pred, nb_acc, nb_roc_auc,nb_f1, nb_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, nb_model)


Accuracy: 0.5433070866141733
ROC-AUC: 0.5
F1-score: 0.7040816326530612


In [164]:
# testing
model = GaussianNB()
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.5357142857142857
ROC-AUC: 0.5064425770308123
F1-score: 0.6904761904761905


In [165]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=10000)
lr_pred, lr_acc, lr_roc_auc,lr_f1, lr_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, lr_model)


Accuracy: 0.47244094488188976
ROC-AUC: 0.4746376811594203
F1-score: 0.4806201550387597


In [166]:
# testing
model = LogisticRegression(max_iter=10000)

model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")

Accuracy: 0.5223214285714286
ROC-AUC: 0.5392156862745098
F1-score: 0.3742690058479532


In [167]:
from sklearn.neighbors import KNeighborsClassifier
grid_params = {'n_neighbors': [1, 2, 3, 4, 5, 10, 20, 50]}
knn_model = KNeighborsClassifier()
knn_pred, knn_acc, knn_roc_auc,knn_f1, knn_feat_imp = cv_and_predict_valid(df_x_dum_train,df_y_train, knn_model, grid_search=True, grid_parameters=grid_params)


KNeighborsClassifier(n_neighbors=10)
Best parameters from grid search: {'n_neighbors': 10}
Accuracy: 0.5275590551181102
ROC-AUC: 0.5363568215892054
F1-score: 0.5


In [168]:
# testing
model = KNeighborsClassifier(n_neighbors=10)
model.fit(df_x_dum_train, df_y_train)
predictions = model.predict(df_x_dum_test)
accuracy = accuracy_score(df_y_test, predictions)
roc_auc = roc_auc_score(df_y_test, predictions)
f1 = f1_score(df_y_test, predictions)
    
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print(f"F1-score: {f1}")
# good

Accuracy: 0.49107142857142855
ROC-AUC: 0.5008403361344539
F1-score: 0.41836734693877553
